# Optimize Initial Conditions
## Yabox

In [1]:
import numpy as np
from datetime import datetime,timedelta
import pandas as pd
from yabox import DE

# Initialize Ray

In [2]:
from environs import Env
env = Env()
env.str("CUDA_DEVICE_ORDER",'PCI_BUS_ID')
env.int("CUDA_VISIBLE_DEVICES",1)
env.int("NUMBA_ENABLE_CUDASIM",1)
env.bool("OMPI_MCA_opal_cuda_support",True)

import os
import ray
MB=1024*1024
GB=MB*1024
ray.shutdown()
ray.init(object_store_memory=1*GB,memory=220*GB,
         lru_evict=True,
         driver_object_store_memory=500*MB,num_gpus=5,num_cpus=1,
         ignore_reinit_error=True) # , include_webui=False)

@ray.remote(num_gpus=1)
def use_gpu():
    print("ray.get_gpu_ids(): {}".format(ray.get_gpu_ids()[0]))
    print("CUDA_VISIBLE_DEVICES: {}".format(os.environ["CUDA_VISIBLE_DEVICES"]))

use_gpu.remote()

2020-10-21 20:39:22,079	INFO resource_spec.py:212 -- Starting Ray with 219.97 GiB memory available for workers and up to 1.0 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-21 20:39:22,389	INFO services.py:1170 -- View the Ray dashboard at localhost:8265


ObjectID(45b95b1c8bd3a9c4ffffffff010000c801000000)

# Load New and Process Data from website data.brasil.io

In [3]:
%reload_ext autoreload
%autoreload 2
import get_data
LoadData=True

if LoadData:
    get_data.get_data()

(pid=14727) ray.get_gpu_ids(): 4
(pid=14727) CUDA_VISIBLE_DEVICES: 4


# Functions to Load Processed Data

In [4]:
def load_confirmed(country,start_date=None):
    df = pd.read_csv('data/time_series_19-covid-Confirmed-country.csv')
    country_df = df[df['Country/Region'] == country]
    if start_date==None:
        return country_df.iloc[0]
    else: 
        return country_df.iloc[0].loc[start_date:]

def load_recovered(country,start_date=None):
    df = pd.read_csv('data/time_series_19-covid-Recovered-country.csv')
    country_df = df[df['Country/Region'] == country]
    if start_date==None:
        return country_df.iloc[0]
    else: 
        return country_df.iloc[0].loc[start_date:]

def load_dead(country,start_date=None):
    df = pd.read_csv('data/time_series_19-covid-Deaths-country.csv')
    country_df = df[df['Country/Region'] == country]
    if start_date==None:
        return country_df.iloc[0]
    else: 
        return country_df.iloc[0].loc[start_date:]

# Load solver

In [5]:
%reload_ext autoreload
%autoreload 2
import LearnerICRayNoLoadBH_v3 as L 

# Data for Countries

In [6]:
dfparam = pd.read_csv("data/param.csv")
countries=dfparam.country
popEst = pd.read_csv("data/WPP2019_TotalPopulationBySex.csv")
popEst['popTotal']=pd.to_numeric(popEst.PopTotal, errors='coerce')

for country in countries:
    if country=="US":
        country2="United States of America"    
    else:
        country2=country
    dfparam.loc[dfparam.country==country,'popTotal']=popEst.loc[popEst.Location==country2].loc[popEst.Time==2020].iloc[0,8]*1000
    dfparam.loc[dfparam.country==country,'s0']=popEst.loc[popEst.Location==country2].loc[popEst.Time==2020].iloc[0,8]*1000
    
display(dfparam)
    

,country,start-date,prediction-range,s0,e0,a0,i0,r0,d0,START,WCASES,WREC,WDTH,popTotal
0,Brazil,3/2/20,200,2.125594e+08,0.0001,0.0001,200,100,50,50,0.15,0.05,0.8,2.125594e+08
1,China,1/28/20,200,1.439324e+09,0.0001,0.0001,200,100,50,50,0.15,0.05,0.8,1.439324e+09
2,Italy,2/28/20,200,6.046183e+07,0.0001,0.0001,200,100,50,50,0.15,0.05,0.8,6.046183e+07
3,US,2/20/20,200,3.310026e+08,0.0001,0.0001,200,100,50,50,0.15,0.05,0.8,3.310026e+08
4,India,3/10/20,200,1.380004e+09,0.0001,0.0001,200,100,50,50,0.15,0.05,0.8,1.380004e+09


# Functions for Optimization

In [7]:
from scipy.integrate import odeint
import sys
import io
import gc

def create_f(country,e0,a0,date, predict_range, version):
                
    def fobjective(point):
        
        dead=  load_dead(country,date)
        recovered = load_recovered(country,date)
        data = load_confirmed(country,date)-recovered-dead
        cleanRecovered=False
        s0, deltaDate, i0, d0, r0, startNCases, weigthCases, weigthRecov, weightDeath = point
        end_date=datetime.strptime(date, "%m/%d/%y") + timedelta(days=deltaDate)
        f=L.Learner.remote(country, end_date.strftime("%m/%d/%y"), predict_range,\
                           s0, e0, a0, i0, r0, d0, startNCases, weigthCases, weigthRecov, weightDeath, \
                           cleanRecovered, version, data, dead, recovered, savedata=False)
        result = f.train.remote() 
        result = ray.get(result) 

        del end_date,cleanRecovered, data, dead, point,f         

        gc.collect()

        return result
    return fobjective

In [8]:
@ray.remote(memory=50 * 1024 * 1024, max_calls=1)
def opt(country,s0,i0,e0,a0,r0,d0,wcases,wrec,wdeath,date,startNCases, 
        predict_range, version):

    bounds=[(20e3,s0),(-2,5),(0,2000), (0,2000),(0,2000),(0,2000),\
              (0.15,1.0),(0.001,0.5),(0.001,0.5)]
    maxiterations=500
    f=create_f(country,e0,a0,date, predict_range, version)
    de = DE(f, bounds, maxiters=maxiterations)
    for step in de.geniterator():
        try:
            idx = step.best_idx
            norm_vector = step.population[idx]
            best_params = de.denormalize([norm_vector])
            del norm_vector, idx
        except:
            print("error in function evaluation")
    p=best_params[0]
    del f, bounds, data, dead,best_params
    
    return p

# Main Code

In [9]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return true;}

<IPython.core.display.Javascript object>

In [10]:
countries=dfparam.country
display(countries)
allCountries=True
version="009"
gc.enable()

optimal=[]
if allCountries:
    for country in countries:
        #remove previous history file
        strFile='./results/history_'+country+version+'.csv'
        if os.path.isfile(strFile):
            os.remove(strFile)
        query = dfparam.query('country == "{}"'.format(country)).reset_index()
        parameters = np.array(query.iloc[:, 2:])[0]
        date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,wcases,wrec,wdth, pop = parameters
        optimal.append(opt.remote(country,s0,i0,e0,a0,r0,d0,wcases,wrec,wdth,date,startNCases, 
                                            predict_range, version))        
else:
    country = "Brazil" 
    #remove previous history file
    strFile='./results/history_'+country+version+'.csv'
    if os.path.isfile(strFile):
        os.remove(strFile)
    query = dfparam.query('country == "{}"'.format(country)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    date,predict_range,s0,e0,a0,i0,r0,d0,startNCases,wcases,wrec,wdth, pop = parameters
    optimal.append(opt.remote(country,s0,i0,e0,a0,r0,d0,wcases,wrec,wdth,date,startNCases, 
                                        predict_range, version))            

0    Brazil
1     China
2     Italy
3        US
4     India
Name: country, dtype: object

In [ ]:
optimal=ray.get(optimal)


2020-10-21 20:39:30,451	WARNING worker.py:1090 -- WARNING: 3 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-10-21 20:39:31,374	WARNING worker.py:1090 -- WARNING: 4 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-10-21 20:39:31,771	WARNING worker.py:1090 -- WARNING: 5 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.
2020-10-21 20:39:32,697	WARNING worker.py:1090 -- WARNING: 6 PYTHON workers have been started. This could be a resu

(pid=14839) basinhopping step 0: f 5.48444e+09
(pid=14839) basinhopping step 1: f 5.48444e+09 trial_f 5.48444e+09 accepted 1  lowest_f 5.48444e+09
(pid=14839) basinhopping step 2: f 5.48444e+09 trial_f 5.48444e+09 accepted 1  lowest_f 5.48444e+09
(pid=14839) basinhopping step 3: f 5.48444e+09 trial_f 5.48444e+09 accepted 1  lowest_f 5.48444e+09


2020-10-21 20:40:59,522	WARNING worker.py:1090 -- WARNING: 10 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=14891) basinhopping step 0: f 6.66981e+11
(pid=14891) basinhopping step 1: f 6.66981e+11 trial_f 6.66981e+11 accepted 1  lowest_f 6.66981e+11
(pid=14891) basinhopping step 2: f 6.66981e+11 trial_f 6.67052e+11 accepted 0  lowest_f 6.66981e+11
(pid=14891) basinhopping step 3: f 6.66981e+11 trial_f 6.66981e+11 accepted 1  lowest_f 6.66981e+11


2020-10-21 20:41:29,353	WARNING worker.py:1090 -- WARNING: 10 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


...

2020-10-22 15:28:27,059	WARNING worker.py:1090 -- WARNING: 10 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=30489) basinhopping step 0: f 1.5391e+12
(pid=30489) basinhopping step 1: f 1.5391e+12 trial_f 1.54373e+12 accepted 0  lowest_f 1.5391e+12
(pid=30489) basinhopping step 2: f 1.52737e+12 trial_f 1.52737e+12 accepted 1  lowest_f 1.52737e+12
(pid=30489) found new global minimum on step 2 with function value 1.52737e+12
(pid=30489) basinhopping step 3: f 1.5146e+12 trial_f 1.5146e+12 accepted 1  lowest_f 1.5146e+12
(pid=30489) found new global minimum on step 3 with function value 1.5146e+12


2020-10-22 15:29:03,073	WARNING worker.py:1090 -- WARNING: 10 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


(pid=30447) basinhopping step 0: f 1.37199e+11
(pid=30447) basinhopping step 1: f 1.33407e+11 trial_f 1.33407e+11 accepted 1  lowest_f 1.33407e+11
(pid=30447) found new global minimum on step 1 with function value 1.33407e+11
(pid=30447) basinhopping step 2: f 1.31742e+11 trial_f 1.31742e+11 accepted 1  lowest_f 1.31742e+11
(pid=30447) found new global minimum on step 2 with function value 1.31742e+11
(pid=30447) basinhopping step 3: f 1.31742e+11 trial_f 1.33636e+11 accepted 0  lowest_f 1.31742e+11


2020-10-22 15:29:20,201	WARNING worker.py:1090 -- WARNING: 10 PYTHON workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


In [ ]:
for i in range(0,len(countries)):    

    country=countries[i]
    query = dfparam.query('country == "{}"'.format(country)).reset_index()
    parameters = np.array(query.iloc[:, 2:])[0]
    startdate,predict_range,s0,e0,a0,i0,r0,d0,startNCases,wcases,wrec = parameters
    Date = datetime.strptime(startdate, "%m/%d/%y")
    end_date = Date + timedelta(days=+int(optimal[i][1]))
    dateStr=end_date.strftime("%m/%d/%y")

    j = query['index'].values[0]
    dfparam.at[j, "s0"] = optimal[i][0]
    dfparam.at[j, "start-date"] = dateStr
    dfparam.at[j, "i0"] = optimal[i][2]
    dfparam.at[j, "WCASES"] = optimal[i][3]
    dfparam.at[j, "WREC"] = optimal[i][4]

    dfparam.to_csv("data/param_optimized_Yabox.csv", sep=",", index=False)
    dfparam
    